<a href="https://colab.research.google.com/github/NUMLDS/stitching-project-Eleanorhhhyxz/blob/main/LoRA_Finetuning_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the LoRA Finetuning pipeline for the sentiment_analyzer agent in the Agentic Advanced RAG system.

# Data Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("torch.cuda.is_available():", torch.cuda.is_available())

torch.cuda.is_available(): False


In [ ]:
device

device(type='cpu')

In [ ]:
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    pipeline
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model, PeftModel
import evaluate

In [ ]:
# Use the same Data Pipeline in the Main_Stitching or Embedding_Upserting_Pipeline code and load the stored filtered_reviews here
filtered_reviews = pd.read_csv("/content/drive/MyDrive/dl_models/GenAI_Stitching/filtered_reviews.csv")
filtered_reviews

<ipython-input-6-4783c450abca>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_reviews = pd.read_csv("/content/drive/MyDrive/Datasets/filtered_reviews.csv")


,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,1741593524,5,1.0,1.0,2,0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,NaN,brown,dry,black,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.0
1,31423088263,1,0.0,NaN,0,0,0,2023-03-21,I bought this lip mask after reading the revie...,Disappointed,NaN,NaN,NaN,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
2,5061282401,5,1.0,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited ...,New Favorite Routine,light,brown,dry,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
3,6083038851,5,1.0,NaN,0,0,0,2023-03-20,I’ve always loved this formula for a long time...,Can't go wrong with any of them,NaN,brown,combination,black,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
4,47056667835,5,1.0,NaN,0,0,0,2023-03-20,"If you have dry cracked lips, this is a must h...",A must have !!!,light,hazel,combination,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280134,2276253200,5,1.0,NaN,0,0,0,2023-03-13,Consider salicylic acid your secret weapon for...,NaN,fair,brown,combination,NaN,P505392,Multi Action Clear Acne Clearing Treatment Lot...,StriVectin,49.0
280135,28013163278,5,1.0,NaN,0,0,0,2023-03-13,I’ve been using this as my only moisturizer fo...,NaN,NaN,blue,combination,blonde,P505392,Multi Action Clear Acne Clearing Treatment Lot...,StriVectin,49.0
280136,1539813076,5,1.0,NaN,0,0,0,2023-03-13,I got breakouts whenever it’s my time of month...,NaN,light,blue,combination,blonde,P505392,Multi Action Clear Acne Clearing Treatment Lot...,StriVectin,49.0
280137,5595682861,5,1.0,NaN,0,0,0,2023-03-13,I love this!!! I don’t get actual acne just an...,NaN,fair,hazel,oily,NaN,P505392,Multi Action Clear Acne Clearing Treatment Lot...,StriVectin,49.0


In [ ]:
from datasets import load_dataset, Dataset
data = filtered_reviews
# transform the ratings into the ground truth of sentiment labels
def rating_to_sentiment(r):
    if r in [1, 2]:
        return 0  # negative
    elif r == 3:
        return 1  # neutral
    else:
        return 2  # positive
data["label"] = data["rating"].apply(rating_to_sentiment)
data = data[["review_text", "label"]]
neg_sample = data[data["label"] == 0].sample(n=500, random_state=42)
neu_sample = data[data["label"] == 1].sample(n=500, random_state=42)
pos_sample = data[data["label"] == 2].sample(n=500, random_state=42)
sampled_data = pd.concat([neg_sample, neu_sample, pos_sample])
sampled_data

,review_text,label
261175,This product was gifted and I’ll start by sayi...,0
76610,"Seriously painful - created burning, dryness, ...",0
153031,Brought back because unfortunately my combo sk...,0
78542,"#freeproduct I wanted to love this serum, but ...",0
62315,"Super thick, not a fan of this eye cream at al...",0
...,...,...
146871,This product really worked amazingly for my re...,2
151624,This product feels great on my skin. I can fee...,2
182249,I have tried Estée Lauder in the past and want...,2
92870,I had to come write a review. I’ve recently st...,2


In [ ]:
sampled_data.groupby("label").count()

,review_text
label,
0,500
1,500
2,500


In [ ]:
from sklearn.model_selection import train_test_split
train_val_data, test_data = train_test_split(sampled_data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_val_data, test_size=0.1, random_state=42)
print("Train size:", len(train_data), "Val size:", len(val_data), "Test size:", len(test_data))

Train size: 1215 Val size: 135 Test size: 150


In [ ]:
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

In [ ]:
train_dataset[0]

{'review_text': 'New holy grail!! I wish they had a bigger size available. I have very dry skin so I do use a moisturizer after applying this serum. My skin is noticeably more clear (if I notice a break out and use this, the next day my face is clear). It hydrates and makes me glow!! The packaging is so cute. This brand is a hidden gem!!!  I will continue to repurchase.',
 'label': 2,
 '__index_level_0__': 199879}

In [ ]:
model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3  # we have 3 labels for demonstration: 0=negative, 1=neutral, 2=positive
).to(device)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--prajjwal1--bert-tiny/snapshots/6f75de8b60a9f8a2fdf7b69cbd86d9e64bcb3837/config.json
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.48.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--prajjwal1--bert-tiny/snapshots/6f75de8b60a9f8a2fdf7b69cbd86d9e64bcb3837/vocab.txt


In [ ]:
train_dataset = train_dataset.remove_columns(["__index_level_0__"])
val_dataset = val_dataset.remove_columns(["__index_level_0__"])
test_dataset = test_dataset.remove_columns(["__index_level_0__"])
# Tokenize the Dataset
def tokenize_function(examples):
    return tokenizer(examples["review_text"], truncation=True, padding="max_length", max_length=64)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Remove the original text column
train_dataset = train_dataset.remove_columns(["review_text"])
val_dataset = val_dataset.remove_columns(["review_text"])
test_dataset = test_dataset.remove_columns(["review_text"])

# Rename "label" -> "labels" for Trainer
train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

# Set format to torch
train_dataset.set_format("torch")
val_dataset.set_format("torch")
test_dataset.set_format("torch")

Map:   0%|          | 0/1215 [00:00<?, ? examples/s]

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1215
})

In [ ]:
# Evaluate Base Model Performance on test data
import torch
import evaluate
import numpy as np

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def evaluate_model(model, val_dataset, batch_size=8):
    model.eval()
    model.to(device)
    from torch.utils.data import DataLoader
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            # outputs.logits shape: (batch_size, num_labels)
            logits = outputs.logits
            predictions = logits.argmax(dim=-1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    # Compute Metrics
    acc = accuracy_metric.compute(predictions=all_predictions, references=all_labels)["accuracy"]
    f1 = f1_metric.compute(predictions=all_predictions, references=all_labels, average="weighted")["f1"]
    return {"accuracy": acc, "f1": f1}

results_base = evaluate_model(base_model, test_dataset, batch_size=8)
print("Base Model Results：", results_base)

Base Model Results： {'accuracy': 0.2866666666666667, 'f1': 0.2521687303353312}


# LoRA fine-tuning with PEFT

<h4>Freeze the original model parameters to only train the new LoRA weights</h4>

In [ ]:
base_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
# First freeze the model parameters
for param in base_model.parameters():
    param.requires_grad = False
    # changing 1d parameters such as biases to high precision may help with stability
    if param.ndim == 1:
        param.data = param.data.to(torch.float32)

# helps with memory, but is not necessary
base_model.gradient_checkpointing_enable()

In [ ]:
# prints how many parameters of the model are trainable (not frozen)
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(base_model)

trainable params: 0 || all params: 4425606 || trainable%: 0.0


<h4>Initialize PEFT LoRA configuration</h4>

- r: controls the rank of the low rank adaptation, larger r means more learnable parameters in each LoRA adapter  
- lora_alpha: scaling factor used to balance how much weight is given to the LoRA activations compared to the initial layer
- target_modules: which modules in the model (layers in the Transformer) to adapt during fine-tuning

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["query", "key", "value", "dense", "attention.output.dense"], # Finetune the Attention Layers
    task_type="SEQ_CLS"
)
peft_model = get_peft_model(base_model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 39,299 || all params: 4,425,606 || trainable%: 0.8880


<h4>Setup trainer() object and fine-tune</h4>

In [ ]:
# Set seed for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [ ]:
# Set Evaluation Metrics
import evaluate
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits_tensor = torch.tensor(logits)
    predictions = logits_tensor.argmax(dim=-1)
    # Accuracy
    acc = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    # F1 (macro-averaged across classes)
    # - "macro" treats all classes equally
    # - "weighted" accounts for class imbalance
    # - "micro" is often used for multi-label tasks
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    return {"accuracy": acc, "f1": f1}

In [ ]:
# Set up Training Hyperparameters
training_args = TrainingArguments(
    output_dir="berttiny_lora_checkpoints",
    eval_strategy="epoch", # Evaluate at the end of each epoch
    save_strategy="epoch", # Save a checkpoint at the end of each epoch
    learning_rate=1e-3,
    num_train_epochs=7,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=10, # Log metrics (loss, etc.) every 10 steps
    load_best_model_at_end=True, # Reload the best checkpoint (based on `metric_for_best_model`)
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to=[],
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
<ipython-input-55-1dd16720f949>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
from transformers.utils import logging
logging.set_verbosity_info()
trainer.train()
trainer.save_model("/content/drive/MyDrive/dl_models/finetuned_lora_model")

***** Running training *****
  Num examples = 1,215
  Num Epochs = 7
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 2,128
  Number of trainable parameters = 39,299


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.802100,0.800031,0.651852,0.648001
2,0.795300,0.909079,0.577778,0.579153
3,0.568100,0.916941,0.600000,0.595931
4,0.773300,1.014442,0.562963,0.559326
5,0.656700,0.903321,0.614815,0.607827
6,0.820400,1.094466,0.548148,0.538021
7,0.505200,0.996710,0.644444,0.642253



***** Running Evaluation *****
  Num examples = 135
  Batch size = 4
Saving model checkpoint to berttiny_lora_checkpoints/checkpoint-304
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--prajjwal1--bert-tiny/snapshots/6f75de8b60a9f8a2fdf7b69cbd86d9e64bcb3837/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.48.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

tokenizer config file saved in berttiny_lora_checkpoints/checkpoint-304/tokenizer_config.json
Special tokens file saved in berttiny_lora_checkpoints/chec

# 4.	Evaluate the fine-tuned model

### Sample Outputs from the trained model
Qualitatively, we can see that the finetuned model provides much better summarization to the articles than the previous base model. We can see that the generated outputs are on the right direction of summarizing the key points and significant results of the academic articles.

In [ ]:
from transformers import AutoConfig, AutoTokenizer
model_path = "/content/drive/MyDrive/dl_models/finetuned_lora_model"
#trained_model = AutoModelForSequenceClassification.from_pretrained(model_path)
trained_tokenizer = AutoTokenizer.from_pretrained(model_path)
base_model_for_lora = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
).to(device)
# Add finetuned LoRA model on the top of base model
peft_model = PeftModel.from_pretrained(
    base_model_for_lora,
    "/content/drive/MyDrive/dl_models/finetuned_lora_model"
)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--prajjwal1--bert-tiny/snapshots/6f75de8b60a9f8a2fdf7b69cbd86d9e64bcb3837/config.json
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformer

In [ ]:
# Evaluate the finetuned LoRA model
import torch
import evaluate
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def evaluate_model(model, val_dataset, batch_size=8):
    model.eval()
    model.to(device)
    from torch.utils.data import DataLoader
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            # outputs.logits shape: (batch_size, num_labels)
            logits = outputs.logits
            predictions = logits.argmax(dim=-1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    # Compute Metrics
    acc = accuracy_metric.compute(predictions=all_predictions, references=all_labels)["accuracy"]
    f1 = f1_metric.compute(predictions=all_predictions, references=all_labels, average="weighted")["f1"]
    print(all_predictions)
    return {"accuracy": acc, "f1": f1}

results_lora = evaluate_model(peft_model, test_dataset, batch_size=8)
print("Model Results with Fintuned Lora：", results_lora)

[np.int64(1), np.int64(2), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(2), np.int64(1), np.int64(2), np.int64(0), np.int64(2), np.int64(2), np.int64(1), np.int64(0), np.int64(0), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(0), np.int64(2), np.int64(1), np.int64(2), np.int64(1), np.int64(1), np.int64(2), np.int64(2), np.int64(2), np.int64(0), np.int64(0), np.int64(2), np.int64(0), np.int64(2), np.int64(0), np.int64(2), np.int64(0), np.int64(2), np.int64(0), np.int64(2), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(2), np.int64(2), np.int64(1), np.int64(1), np.int64(2), np.int64(2), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(2), np.int64(1), np.int64(0), np.int64(2), np.int64(2), np.int64(0), np.int64(0), np.int64(0), np.int64(2), np.int64(1), np.int64(1), np.int64(2), np.int64(0)